{
 "cells": [
  {
   "cell_type": "markdown",
   "id": "56c4e6d1",
   "metadata": {},
   "source": [
    "## Weighted stats\n",
    "- This notebook is made in the Programming for Data Analytics lecture on weighted descriptive statistics.\n",
    "- We are looking at popultions in each county of ireland\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1,
   "id": "e71b0552",
   "metadata": {},
   "outputs": [],
   "source": [
    "import pandas as pd"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 2,
   "id": "d0e6aaf4",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>STATISTIC</th>\n",
       "      <th>Statistic Label</th>\n",
       "      <th>TLIST(A1)</th>\n",
       "      <th>CensusYear</th>\n",
       "      <th>C02199V02655</th>\n",
       "      <th>Sex</th>\n",
       "      <th>C02076V03371</th>\n",
       "      <th>Single Year of Age</th>\n",
       "      <th>C03789V04537</th>\n",
       "      <th>Administrative Counties</th>\n",
       "      <th>UNIT</th>\n",
       "      <th>VALUE</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>9789</th>\n",
       "      <td>FY006AC01</td>\n",
       "      <td>Population</td>\n",
       "      <td>2022</td>\n",
       "      <td>2022</td>\n",
       "      <td>2</td>\n",
       "      <td>Female</td>\n",
       "      <td>650</td>\n",
       "      <td>100 years and over</td>\n",
       "      <td>2ae19629-149d-13a3-e055-000000000001</td>\n",
       "      <td>Cavan County Council</td>\n",
       "      <td>Number</td>\n",
       "      <td>12</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>9790</th>\n",
       "      <td>FY006AC01</td>\n",
       "      <td>Population</td>\n",
       "      <td>2022</td>\n",
       "      <td>2022</td>\n",
       "      <td>2</td>\n",
       "      <td>Female</td>\n",
       "      <td>650</td>\n",
       "      <td>100 years and over</td>\n",
       "      <td>2ae19629-14a4-13a3-e055-000000000001</td>\n",
       "      <td>Donegal County Council</td>\n",
       "      <td>Number</td>\n",
       "      <td>31</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>9791</th>\n",
       "      <td>FY006AC01</td>\n",
       "      <td>Population</td>\n",
       "      <td>2022</td>\n",
       "      <td>2022</td>\n",
       "      <td>2</td>\n",
       "      <td>Female</td>\n",
       "      <td>650</td>\n",
       "      <td>100 years and over</td>\n",
       "      <td>2ae19629-1495-13a3-e055-000000000001</td>\n",
       "      <td>Monaghan County Council</td>\n",
       "      <td>Number</td>\n",
       "      <td>7</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "</div>"
      ],
      "text/plain": [
       "      STATISTIC Statistic Label  TLIST(A1)  CensusYear C02199V02655     Sex  \\\n",
       "9789  FY006AC01      Population       2022        2022            2  Female   \n",
       "9790  FY006AC01      Population       2022        2022            2  Female   \n",
       "9791  FY006AC01      Population       2022        2022            2  Female   \n",
       "\n",
       "     C02076V03371  Single Year of Age                          C03789V04537  \\\n",
       "9789          650  100 years and over  2ae19629-149d-13a3-e055-000000000001   \n",
       "9790          650  100 years and over  2ae19629-14a4-13a3-e055-000000000001   \n",
       "9791          650  100 years and over  2ae19629-1495-13a3-e055-000000000001   \n",
       "\n",
       "      Administrative Counties    UNIT  VALUE  \n",
       "9789     Cavan County Council  Number     12  \n",
       "9790   Donegal County Council  Number     31  \n",
       "9791  Monaghan County Council  Number      7  "
      ]
     },
     "execution_count": 2,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "url = \"https://ws.cso.ie/public/api.restful/PxStat.Data.Cube_API.ReadDataset/FY006A/CSV/1.0/en\"\n",
    "df = pd.read_csv(url)\n",
    "df.tail(3)"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "1979172c",
   "metadata": {},
   "source": [
    "ok I notice that this has all the data for each gender I want to get rid of that"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 3,
   "id": "8ec2b205",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>STATISTIC</th>\n",
       "      <th>Statistic Label</th>\n",
       "      <th>TLIST(A1)</th>\n",
       "      <th>CensusYear</th>\n",
       "      <th>C02199V02655</th>\n",
       "      <th>Sex</th>\n",
       "      <th>C02076V03371</th>\n",
       "      <th>Single Year of Age</th>\n",
       "      <th>C03789V04537</th>\n",
       "      <th>Administrative Counties</th>\n",
       "      <th>UNIT</th>\n",
       "      <th>VALUE</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>3261</th>\n",
       "      <td>FY006AC01</td>\n",
       "      <td>Population</td>\n",
       "      <td>2022</td>\n",
       "      <td>2022</td>\n",
       "      <td>-</td>\n",
       "      <td>Both sexes</td>\n",
       "      <td>650</td>\n",
       "      <td>100 years and over</td>\n",
       "      <td>2ae19629-149d-13a3-e055-000000000001</td>\n",
       "      <td>Cavan County Council</td>\n",
       "      <td>Number</td>\n",
       "      <td>18</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>3262</th>\n",
       "      <td>FY006AC01</td>\n",
       "      <td>Population</td>\n",
       "      <td>2022</td>\n",
       "      <td>2022</td>\n",
       "      <td>-</td>\n",
       "      <td>Both sexes</td>\n",
       "      <td>650</td>\n",
       "      <td>100 years and over</td>\n",
       "      <td>2ae19629-14a4-13a3-e055-000000000001</td>\n",
       "      <td>Donegal County Council</td>\n",
       "      <td>Number</td>\n",
       "      <td>33</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>3263</th>\n",
       "      <td>FY006AC01</td>\n",
       "      <td>Population</td>\n",
       "      <td>2022</td>\n",
       "      <td>2022</td>\n",
       "      <td>-</td>\n",
       "      <td>Both sexes</td>\n",
       "      <td>650</td>\n",
       "      <td>100 years and over</td>\n",
       "      <td>2ae19629-1495-13a3-e055-000000000001</td>\n",
       "      <td>Monaghan County Council</td>\n",
       "      <td>Number</td>\n",
       "      <td>8</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "</div>"
      ],
      "text/plain": [
       "      STATISTIC Statistic Label  TLIST(A1)  CensusYear C02199V02655  \\\n",
       "3261  FY006AC01      Population       2022        2022            -   \n",
       "3262  FY006AC01      Population       2022        2022            -   \n",
       "3263  FY006AC01      Population       2022        2022            -   \n",
       "\n",
       "             Sex C02076V03371  Single Year of Age  \\\n",
       "3261  Both sexes          650  100 years and over   \n",
       "3262  Both sexes          650  100 years and over   \n",
       "3263  Both sexes          650  100 years and over   \n",
       "\n",
       "                              C03789V04537  Administrative Counties    UNIT  \\\n",
       "3261  2ae19629-149d-13a3-e055-000000000001     Cavan County Council  Number   \n",
       "3262  2ae19629-14a4-13a3-e055-000000000001   Donegal County Council  Number   \n",
       "3263  2ae19629-1495-13a3-e055-000000000001  Monaghan County Council  Number   \n",
       "\n",
       "      VALUE  \n",
       "3261     18  \n",
       "3262     33  \n",
       "3263      8  "
      ]
     },
     "execution_count": 3,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "df = df[df[\"Sex\"] != \"Female\"]\n",
    "df = df[df[\"Sex\"] != \"Male\"]\n",
    "df.tail(3)"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "9ddd7162",
   "metadata": {},
   "source": [
    "Now I can do the code that is the prep for analysis.py  \n",
    "I just noticed that there were more columns so I need to get their names"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 4,
   "id": "d6d72e19",
   "metadata": {
    "scrolled": true
   },
   "outputs": [
    {
     "data": {
      "text/plain": [
       "['STATISTIC',\n",
       " 'Statistic Label',\n",
       " 'TLIST(A1)',\n",
       " 'CensusYear',\n",
       " 'C02199V02655',\n",
       " 'Sex',\n",
       " 'C02076V03371',\n",
       " 'Single Year of Age',\n",
       " 'C03789V04537',\n",
       " 'Administrative Counties',\n",
       " 'UNIT',\n",
       " 'VALUE']"
      ]
     },
     "execution_count": 4,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "headers = df.columns.tolist()\n",
    "headers"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 5,
   "id": "08f6cda0",
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "<class 'pandas.core.frame.DataFrame'>\n",
      "Int64Index: 3232 entries, 32 to 3263\n",
      "Data columns (total 3 columns):\n",
      " #   Column                   Non-Null Count  Dtype \n",
      "---  ------                   --------------  ----- \n",
      " 0   Single Year of Age       3232 non-null   int64 \n",
      " 1   Administrative Counties  3232 non-null   object\n",
      " 2   VALUE                    3232 non-null   int64 \n",
      "dtypes: int64(2), object(1)\n",
      "memory usage: 101.0+ KB\n"
     ]
    }
   ],
   "source": [
    "drop_col_list = ['STATISTIC', 'Statistic Label','TLIST(A1)','CensusYear','C02199V02655','Sex','C02076V03371','C03789V04537','UNIT']\n",
    "df.drop(columns=drop_col_list, inplace=True)\n",
    "df = df[df[\"Single Year of Age\"] != \"All ages\"]\n",
    "df['Single Year of Age'] = df['Single Year of Age'].str.replace('Under 1 year', '0')\n",
    "df['Single Year of Age'] = df['Single Year of Age'].str.replace('\\D', '', regex=True)\n",
    "\n",
    "df['Single Year of Age']=df['Single Year of Age'].astype('int64')\n",
    "df['VALUE']=df['VALUE'].astype('int64')\n",
    "#print (df.head(3))\n",
    "df.info()\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 6,
   "id": "ce078e65",
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Administrative Counties  Carlow County Council  Cavan County Council  \\\n",
      "Single Year of Age                                                     \n",
      "0                                          699                  1006   \n",
      "1                                          649                  1004   \n",
      "2                                          689                  1061   \n",
      "\n",
      "Administrative Counties  Clare County Council  Cork City Council  \\\n",
      "Single Year of Age                                                 \n",
      "0                                        1377               2283   \n",
      "1                                        1360               2284   \n",
      "2                                        1473               2306   \n",
      "\n",
      "Administrative Counties  Cork County Council  Donegal County Council  \\\n",
      "Single Year of Age                                                     \n",
      "0                                       4190                    1797   \n",
      "1                                       4115                    1881   \n",
      "2                                       4290                    1971   \n",
      "\n",
      "Administrative Counties  Dublin City Council  \\\n",
      "Single Year of Age                             \n",
      "0                                       6213   \n",
      "1                                       5699   \n",
      "2                                       5830   \n",
      "\n",
      "Administrative Counties  Dún Laoghaire Rathdown County Council  \\\n",
      "Single Year of Age                                               \n",
      "0                                                         2457   \n",
      "1                                                         2437   \n",
      "2                                                         2489   \n",
      "\n",
      "Administrative Counties  Fingal County Council  Galway City Council  ...  \\\n",
      "Single Year of Age                                                   ...   \n",
      "0                                         4009                  792  ...   \n",
      "1                                         3894                  757  ...   \n",
      "2                                         4308                  792  ...   \n",
      "\n",
      "Administrative Counties  Monaghan County Council  Offaly County Council  \\\n",
      "Single Year of Age                                                        \n",
      "0                                            868                    935   \n",
      "1                                            821                    937   \n",
      "2                                            870                    945   \n",
      "\n",
      "Administrative Counties  Roscommon County Council  Sligo County Council  \\\n",
      "Single Year of Age                                                        \n",
      "0                                             748                   743   \n",
      "1                                             764                   766   \n",
      "2                                             893                   764   \n",
      "\n",
      "Administrative Counties  South Dublin County Council  \\\n",
      "Single Year of Age                                     \n",
      "0                                               3544   \n",
      "1                                               3673   \n",
      "2                                               3734   \n",
      "\n",
      "Administrative Counties  Tipperary County Council  \\\n",
      "Single Year of Age                                  \n",
      "0                                            1932   \n",
      "1                                            1808   \n",
      "2                                            1836   \n",
      "\n",
      "Administrative Counties  Waterford City & County Council  \\\n",
      "Single Year of Age                                         \n",
      "0                                                   1340   \n",
      "1                                                   1266   \n",
      "2                                                   1329   \n",
      "\n",
      "Administrative Counties  Westmeath County Council  Wexford County Council  \\\n",
      "Single Year of Age                                                          \n",
      "0                                            1129                    1792   \n",
      "1                                            1158                    1724   \n",
      "2                                            1108                    1835   \n",
      "\n",
      "Administrative Counties  Wicklow County Council  \n",
      "Single Year of Age                               \n",
      "0                                          1710  \n",
      "1                                          1682  \n",
      "2                                          1796  \n",
      "\n",
      "[3 rows x 32 columns]\n"
     ]
    }
   ],
   "source": [
    "df_anal = pd.pivot_table(df, 'VALUE',\"Single Year of Age\",\"Administrative Counties\")\n",
    "print (df_anal.head(3))\n",
    "# write out the entire file to local machine\n",
    "df_anal.to_csv(\"population_for_analysis.csv\")"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "ff892a81",
   "metadata": {},
   "source": [
    "## Now we can do weighted descriptive statistics"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 7,
   "id": "9161be9a",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "'Carlow County Council'"
      ]
     },
     "execution_count": 7,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "headers = list(df_anal.columns)\n",
    "district = headers[0]\n",
    "district"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "deddeb8e",
   "metadata": {},
   "source": [
    "Weighted mean is sum(age*population at age) / sum (populations at age)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 11,
   "id": "407110fb",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "61968"
      ]
     },
     "execution_count": 11,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "number_people = df_anal[district].sum()\n",
    "number_people\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 12,
   "id": "b00affc4",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th>Administrative Counties</th>\n",
       "      <th>Carlow County Council</th>\n",
       "      <th>Cavan County Council</th>\n",
       "      <th>Clare County Council</th>\n",
       "      <th>Cork City Council</th>\n",
       "      <th>Cork County Council</th>\n",
       "      <th>Donegal County Council</th>\n",
       "      <th>Dublin City Council</th>\n",
       "      <th>Dún Laoghaire Rathdown County Council</th>\n",
       "      <th>Fingal County Council</th>\n",
       "      <th>Galway City Council</th>\n",
       "      <th>...</th>\n",
       "      <th>Monaghan County Council</th>\n",
       "      <th>Offaly County Council</th>\n",
       "      <th>Roscommon County Council</th>\n",
       "      <th>Sligo County Council</th>\n",
       "      <th>South Dublin County Council</th>\n",
       "      <th>Tipperary County Council</th>\n",
       "      <th>Waterford City &amp; County Council</th>\n",
       "      <th>Westmeath County Council</th>\n",
       "      <th>Wexford County Council</th>\n",
       "      <th>Wicklow County Council</th>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>Single Year of Age</th>\n",
       "      <th></th>\n",
       "      <th></th>\n",
       "      <th></th>\n",
       "      <th></th>\n",
       "      <th></th>\n",
       "      <th></th>\n",
       "      <th></th>\n",
       "      <th></th>\n",
       "      <th></th>\n",
       "      <th></th>\n",
       "      <th></th>\n",
       "      <th></th>\n",
       "      <th></th>\n",
       "      <th></th>\n",
       "      <th></th>\n",
       "      <th></th>\n",
       "      <th></th>\n",
       "      <th></th>\n",
       "      <th></th>\n",
       "      <th></th>\n",
       "      <th></th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>0</th>\n",
       "      <td>699</td>\n",
       "      <td>1006</td>\n",
       "      <td>1377</td>\n",
       "      <td>2283</td>\n",
       "      <td>4190</td>\n",
       "      <td>1797</td>\n",
       "      <td>6213</td>\n",
       "      <td>2457</td>\n",
       "      <td>4009</td>\n",
       "      <td>792</td>\n",
       "      <td>...</td>\n",
       "      <td>868</td>\n",
       "      <td>935</td>\n",
       "      <td>748</td>\n",
       "      <td>743</td>\n",
       "      <td>3544</td>\n",
       "      <td>1932</td>\n",
       "      <td>1340</td>\n",
       "      <td>1129</td>\n",
       "      <td>1792</td>\n",
       "      <td>1710</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>1</th>\n",
       "      <td>649</td>\n",
       "      <td>1004</td>\n",
       "      <td>1360</td>\n",
       "      <td>2284</td>\n",
       "      <td>4115</td>\n",
       "      <td>1881</td>\n",
       "      <td>5699</td>\n",
       "      <td>2437</td>\n",
       "      <td>3894</td>\n",
       "      <td>757</td>\n",
       "      <td>...</td>\n",
       "      <td>821</td>\n",
       "      <td>937</td>\n",
       "      <td>764</td>\n",
       "      <td>766</td>\n",
       "      <td>3673</td>\n",
       "      <td>1808</td>\n",
       "      <td>1266</td>\n",
       "      <td>1158</td>\n",
       "      <td>1724</td>\n",
       "      <td>1682</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>2</th>\n",
       "      <td>689</td>\n",
       "      <td>1061</td>\n",
       "      <td>1473</td>\n",
       "      <td>2306</td>\n",
       "      <td>4290</td>\n",
       "      <td>1971</td>\n",
       "      <td>5830</td>\n",
       "      <td>2489</td>\n",
       "      <td>4308</td>\n",
       "      <td>792</td>\n",
       "      <td>...</td>\n",
       "      <td>870</td>\n",
       "      <td>945</td>\n",
       "      <td>893</td>\n",
       "      <td>764</td>\n",
       "      <td>3734</td>\n",
       "      <td>1836</td>\n",
       "      <td>1329</td>\n",
       "      <td>1108</td>\n",
       "      <td>1835</td>\n",
       "      <td>1796</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>3</th>\n",
       "      <td>754</td>\n",
       "      <td>1085</td>\n",
       "      <td>1468</td>\n",
       "      <td>2294</td>\n",
       "      <td>4437</td>\n",
       "      <td>2074</td>\n",
       "      <td>5580</td>\n",
       "      <td>2613</td>\n",
       "      <td>4384</td>\n",
       "      <td>799</td>\n",
       "      <td>...</td>\n",
       "      <td>900</td>\n",
       "      <td>1009</td>\n",
       "      <td>859</td>\n",
       "      <td>825</td>\n",
       "      <td>3856</td>\n",
       "      <td>2048</td>\n",
       "      <td>1476</td>\n",
       "      <td>1148</td>\n",
       "      <td>1864</td>\n",
       "      <td>1862</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>4</th>\n",
       "      <td>745</td>\n",
       "      <td>1047</td>\n",
       "      <td>1505</td>\n",
       "      <td>2243</td>\n",
       "      <td>4620</td>\n",
       "      <td>2089</td>\n",
       "      <td>5624</td>\n",
       "      <td>2658</td>\n",
       "      <td>4422</td>\n",
       "      <td>830</td>\n",
       "      <td>...</td>\n",
       "      <td>873</td>\n",
       "      <td>1007</td>\n",
       "      <td>833</td>\n",
       "      <td>820</td>\n",
       "      <td>3870</td>\n",
       "      <td>2072</td>\n",
       "      <td>1390</td>\n",
       "      <td>1208</td>\n",
       "      <td>1941</td>\n",
       "      <td>1897</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>...</th>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "      <td>...</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>96</th>\n",
       "      <td>11</td>\n",
       "      <td>23</td>\n",
       "      <td>40</td>\n",
       "      <td>54</td>\n",
       "      <td>84</td>\n",
       "      <td>48</td>\n",
       "      <td>200</td>\n",
       "      <td>95</td>\n",
       "      <td>48</td>\n",
       "      <td>19</td>\n",
       "      <td>...</td>\n",
       "      <td>17</td>\n",
       "      <td>19</td>\n",
       "      <td>28</td>\n",
       "      <td>21</td>\n",
       "      <td>41</td>\n",
       "      <td>41</td>\n",
       "      <td>26</td>\n",
       "      <td>24</td>\n",
       "      <td>42</td>\n",
       "      <td>38</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>97</th>\n",
       "      <td>11</td>\n",
       "      <td>13</td>\n",
       "      <td>31</td>\n",
       "      <td>39</td>\n",
       "      <td>65</td>\n",
       "      <td>27</td>\n",
       "      <td>115</td>\n",
       "      <td>57</td>\n",
       "      <td>51</td>\n",
       "      <td>12</td>\n",
       "      <td>...</td>\n",
       "      <td>9</td>\n",
       "      <td>16</td>\n",
       "      <td>13</td>\n",
       "      <td>22</td>\n",
       "      <td>38</td>\n",
       "      <td>33</td>\n",
       "      <td>20</td>\n",
       "      <td>23</td>\n",
       "      <td>37</td>\n",
       "      <td>25</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>98</th>\n",
       "      <td>7</td>\n",
       "      <td>8</td>\n",
       "      <td>15</td>\n",
       "      <td>23</td>\n",
       "      <td>47</td>\n",
       "      <td>15</td>\n",
       "      <td>81</td>\n",
       "      <td>45</td>\n",
       "      <td>21</td>\n",
       "      <td>15</td>\n",
       "      <td>...</td>\n",
       "      <td>11</td>\n",
       "      <td>8</td>\n",
       "      <td>7</td>\n",
       "      <td>12</td>\n",
       "      <td>17</td>\n",
       "      <td>21</td>\n",
       "      <td>17</td>\n",
       "      <td>11</td>\n",
       "      <td>22</td>\n",
       "      <td>16</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>99</th>\n",
       "      <td>7</td>\n",
       "      <td>6</td>\n",
       "      <td>14</td>\n",
       "      <td>24</td>\n",
       "      <td>25</td>\n",
       "      <td>18</td>\n",
       "      <td>64</td>\n",
       "      <td>32</td>\n",
       "      <td>22</td>\n",
       "      <td>4</td>\n",
       "      <td>...</td>\n",
       "      <td>5</td>\n",
       "      <td>4</td>\n",
       "      <td>9</td>\n",
       "      <td>10</td>\n",
       "      <td>17</td>\n",
       "      <td>10</td>\n",
       "      <td>19</td>\n",
       "      <td>6</td>\n",
       "      <td>21</td>\n",
       "      <td>8</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>100</th>\n",
       "      <td>3</td>\n",
       "      <td>18</td>\n",
       "      <td>19</td>\n",
       "      <td>26</td>\n",
       "      <td>54</td>\n",
       "      <td>33</td>\n",
       "      <td>100</td>\n",
       "      <td>55</td>\n",
       "      <td>21</td>\n",
       "      <td>8</td>\n",
       "      <td>...</td>\n",
       "      <td>8</td>\n",
       "      <td>5</td>\n",
       "      <td>12</td>\n",
       "      <td>12</td>\n",
       "      <td>36</td>\n",
       "      <td>23</td>\n",
       "      <td>19</td>\n",
       "      <td>12</td>\n",
       "      <td>16</td>\n",
       "      <td>18</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "<p>101 rows × 32 columns</p>\n",
       "</div>"
      ],
      "text/plain": [
       "Administrative Counties  Carlow County Council  Cavan County Council  \\\n",
       "Single Year of Age                                                     \n",
       "0                                          699                  1006   \n",
       "1                                          649                  1004   \n",
       "2                                          689                  1061   \n",
       "3                                          754                  1085   \n",
       "4                                          745                  1047   \n",
       "...                                        ...                   ...   \n",
       "96                                          11                    23   \n",
       "97                                          11                    13   \n",
       "98                                           7                     8   \n",
       "99                                           7                     6   \n",
       "100                                          3                    18   \n",
       "\n",
       "Administrative Counties  Clare County Council  Cork City Council  \\\n",
       "Single Year of Age                                                 \n",
       "0                                        1377               2283   \n",
       "1                                        1360               2284   \n",
       "2                                        1473               2306   \n",
       "3                                        1468               2294   \n",
       "4                                        1505               2243   \n",
       "...                                       ...                ...   \n",
       "96                                         40                 54   \n",
       "97                                         31                 39   \n",
       "98                                         15                 23   \n",
       "99                                         14                 24   \n",
       "100                                        19                 26   \n",
       "\n",
       "Administrative Counties  Cork County Council  Donegal County Council  \\\n",
       "Single Year of Age                                                     \n",
       "0                                       4190                    1797   \n",
       "1                                       4115                    1881   \n",
       "2                                       4290                    1971   \n",
       "3                                       4437                    2074   \n",
       "4                                       4620                    2089   \n",
       "...                                      ...                     ...   \n",
       "96                                        84                      48   \n",
       "97                                        65                      27   \n",
       "98                                        47                      15   \n",
       "99                                        25                      18   \n",
       "100                                       54                      33   \n",
       "\n",
       "Administrative Counties  Dublin City Council  \\\n",
       "Single Year of Age                             \n",
       "0                                       6213   \n",
       "1                                       5699   \n",
       "2                                       5830   \n",
       "3                                       5580   \n",
       "4                                       5624   \n",
       "...                                      ...   \n",
       "96                                       200   \n",
       "97                                       115   \n",
       "98                                        81   \n",
       "99                                        64   \n",
       "100                                      100   \n",
       "\n",
       "Administrative Counties  Dún Laoghaire Rathdown County Council  \\\n",
       "Single Year of Age                                               \n",
       "0                                                         2457   \n",
       "1                                                         2437   \n",
       "2                                                         2489   \n",
       "3                                                         2613   \n",
       "4                                                         2658   \n",
       "...                                                        ...   \n",
       "96                                                          95   \n",
       "97                                                          57   \n",
       "98                                                          45   \n",
       "99                                                          32   \n",
       "100                                                         55   \n",
       "\n",
       "Administrative Counties  Fingal County Council  Galway City Council  ...  \\\n",
       "Single Year of Age                                                   ...   \n",
       "0                                         4009                  792  ...   \n",
       "1                                         3894                  757  ...   \n",
       "2                                         4308                  792  ...   \n",
       "3                                         4384                  799  ...   \n",
       "4                                         4422                  830  ...   \n",
       "...                                        ...                  ...  ...   \n",
       "96                                          48                   19  ...   \n",
       "97                                          51                   12  ...   \n",
       "98                                          21                   15  ...   \n",
       "99                                          22                    4  ...   \n",
       "100                                         21                    8  ...   \n",
       "\n",
       "Administrative Counties  Monaghan County Council  Offaly County Council  \\\n",
       "Single Year of Age                                                        \n",
       "0                                            868                    935   \n",
       "1                                            821                    937   \n",
       "2                                            870                    945   \n",
       "3                                            900                   1009   \n",
       "4                                            873                   1007   \n",
       "...                                          ...                    ...   \n",
       "96                                            17                     19   \n",
       "97                                             9                     16   \n",
       "98                                            11                      8   \n",
       "99                                             5                      4   \n",
       "100                                            8                      5   \n",
       "\n",
       "Administrative Counties  Roscommon County Council  Sligo County Council  \\\n",
       "Single Year of Age                                                        \n",
       "0                                             748                   743   \n",
       "1                                             764                   766   \n",
       "2                                             893                   764   \n",
       "3                                             859                   825   \n",
       "4                                             833                   820   \n",
       "...                                           ...                   ...   \n",
       "96                                             28                    21   \n",
       "97                                             13                    22   \n",
       "98                                              7                    12   \n",
       "99                                              9                    10   \n",
       "100                                            12                    12   \n",
       "\n",
       "Administrative Counties  South Dublin County Council  \\\n",
       "Single Year of Age                                     \n",
       "0                                               3544   \n",
       "1                                               3673   \n",
       "2                                               3734   \n",
       "3                                               3856   \n",
       "4                                               3870   \n",
       "...                                              ...   \n",
       "96                                                41   \n",
       "97                                                38   \n",
       "98                                                17   \n",
       "99                                                17   \n",
       "100                                               36   \n",
       "\n",
       "Administrative Counties  Tipperary County Council  \\\n",
       "Single Year of Age                                  \n",
       "0                                            1932   \n",
       "1                                            1808   \n",
       "2                                            1836   \n",
       "3                                            2048   \n",
       "4                                            2072   \n",
       "...                                           ...   \n",
       "96                                             41   \n",
       "97                                             33   \n",
       "98                                             21   \n",
       "99                                             10   \n",
       "100                                            23   \n",
       "\n",
       "Administrative Counties  Waterford City & County Council  \\\n",
       "Single Year of Age                                         \n",
       "0                                                   1340   \n",
       "1                                                   1266   \n",
       "2                                                   1329   \n",
       "3                                                   1476   \n",
       "4                                                   1390   \n",
       "...                                                  ...   \n",
       "96                                                    26   \n",
       "97                                                    20   \n",
       "98                                                    17   \n",
       "99                                                    19   \n",
       "100                                                   19   \n",
       "\n",
       "Administrative Counties  Westmeath County Council  Wexford County Council  \\\n",
       "Single Year of Age                                                          \n",
       "0                                            1129                    1792   \n",
       "1                                            1158                    1724   \n",
       "2                                            1108                    1835   \n",
       "3                                            1148                    1864   \n",
       "4                                            1208                    1941   \n",
       "...                                           ...                     ...   \n",
       "96                                             24                      42   \n",
       "97                                             23                      37   \n",
       "98                                             11                      22   \n",
       "99                                              6                      21   \n",
       "100                                            12                      16   \n",
       "\n",
       "Administrative Counties  Wicklow County Council  \n",
       "Single Year of Age                               \n",
       "0                                          1710  \n",
       "1                                          1682  \n",
       "2                                          1796  \n",
       "3                                          1862  \n",
       "4                                          1897  \n",
       "...                                         ...  \n",
       "96                                           38  \n",
       "97                                           25  \n",
       "98                                           16  \n",
       "99                                            8  \n",
       "100                                          18  \n",
       "\n",
       "[101 rows x 32 columns]"
      ]
     },
     "execution_count": 12,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "df_anal"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 13,
   "id": "76df51bd",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "2371825"
      ]
     },
     "execution_count": 13,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "cumages = df_anal[district].mul(df_anal.index, axis=0).sum()\n",
    "cumages"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 14,
   "id": "8429c793",
   "metadata": {
    "scrolled": true
   },
   "outputs": [
    {
     "data": {
      "text/plain": [
       "38.27499677252776"
      ]
     },
     "execution_count": 14,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "weighted_mean = cumages/number_people\n",
    "weighted_mean"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "93b7f86a",
   "metadata": {},
   "source": [
    "#### Or you can use numpy"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 15,
   "id": "accfeb93",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "38.27499677252776"
      ]
     },
     "execution_count": 15,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "import numpy as np\n",
    "w_mean = np.average(df_anal.index, weights=df_anal[district])\n",
    "w_mean"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "30b296bb",
   "metadata": {},
   "source": [
    "### weighted median"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "de69f1d7",
   "metadata": {},
   "source": [
    "    create a series of the cumlative sums and find the index of the middle value"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 16,
   "id": "6a6c94b9",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "Single Year of Age\n",
       "0        699\n",
       "1       1348\n",
       "2       2037\n",
       "3       2791\n",
       "4       3536\n",
       "       ...  \n",
       "96     61940\n",
       "97     61951\n",
       "98     61958\n",
       "99     61965\n",
       "100    61968\n",
       "Name: Carlow County Council, Length: 101, dtype: int64"
      ]
     },
     "execution_count": 16,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "cumsum = df_anal[district].cumsum()\n",
    "cumsum"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 17,
   "id": "6cecefcf",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "30984.0"
      ]
     },
     "execution_count": 17,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "# sum of all the number of people\n",
    "cutoff = df_anal[district].sum()/2\n",
    "cutoff"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 18,
   "id": "595d8ea3",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "39"
      ]
     },
     "execution_count": 18,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "df_anal[district][cumsum>=cutoff].index[0]"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "cf050b93",
   "metadata": {},
   "source": [
    "## Weighted standard deviation"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "58c2fa02",
   "metadata": {},
   "source": [
    "The is the same formula as the normal standard deviation, just with weights applied to the differenced to the weight mean"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 19,
   "id": "df4ac737",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "306.958515918763"
      ]
     },
     "execution_count": 19,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "# incorrect way\n",
    "df_anal[district].std()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 20,
   "id": "cb52f2f7",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "613.5445544554456"
      ]
     },
     "execution_count": 20,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "# incorrect\n",
    "df_anal[district].mean()"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "5588af73",
   "metadata": {},
   "source": [
    "We need the weighted std"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 21,
   "id": "6b8e8b8e",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "38.27499677252776"
      ]
     },
     "execution_count": 21,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "import numpy as np\n",
    "w_mean = np.average(df_anal.index, weights=df_anal[district])\n",
    "w_mean"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 22,
   "id": "482931fa",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "521.6941450425923"
      ]
     },
     "execution_count": 22,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "w_variance = np.average((df_anal.index - w_mean)**2, weights= df_anal[district])\n",
    "w_variance"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 23,
   "id": "b68421b8",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "22.84062488292718"
      ]
     },
     "execution_count": 23,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "w_std = np.sqrt(w_variance)\n",
    "w_std"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3 (ipykernel)",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.9.7"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}